<a href="https://colab.research.google.com/github/GusevVlad/GusevVlad.github.io/blob/master/%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D0%B0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Разрабатываем собственный инструмент анализа настроений


In [14]:
# !pip install sklearn

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import json
data_json="/content/drive/MyDrive/Shared/train.json"
data_xlsx="/content/drive/MyDrive/Shared/events_new.xlsx"

In [19]:
class CreateDataset():
    def __init__(self, data_json, data_xlsx):
        self.data_json = data_json
        self.data_xlsx = data_xlsx
        
        
    def train(self):
        to_data = {'text' : [], 'polarity' : []}
        transfer = {'negative' : -1, 'neutral' : 0, 'positive' : 1}
        with open(self.data_json, encoding='utf-8') as f:
            data = json.load(f)
        for values in data:
            text = values['text']
            sentiment = transfer[values['sentiment']]
            text = text.replace('\n', '')
            to_data['text'].append(text)
            to_data['polarity'].append(sentiment)
        
        return pd.DataFrame(to_data)
    
    def test(self):
        to_data = {'text' : [], 'polarity' : []}
        dfs = pd.read_excel(self.data_xlsx, sheet_name='ПАТ_САЭС_23.03.2021')
        dfs = dfs[dfs['Unnamed: 6'] == 'Социальные сети'][['Unnamed: 7', 'Unnamed: 8']]
        dfs = dfs[dfs['Unnamed: 8'].notnull()]
        for index in range(len(dfs)):
            values = dfs.iloc[index]
            text = values['Unnamed: 7']
            sentiment = values['Unnamed: 8']
            to_data['text'].append(text)
            to_data['polarity'].append(int(sentiment))
            
        return pd.DataFrame(to_data)

In [20]:
dataset = CreateDataset(data_json, data_xlsx)

In [21]:
df_train = dataset.train()
df_train['text'] = df_train['text'].astype('str')

In [22]:
df_test = dataset.test()
df_test['text'] = df_test['text'].astype('str') 

## Оценим доли положительного/нейтрального/негативного контента в данных

In [23]:
df_train.polarity.value_counts()

 0    4034
 1    2795
-1    1434
Name: polarity, dtype: int64

In [24]:
df_test.polarity.value_counts()

 0    82
-1    23
 1    13
Name: polarity, dtype: int64

In [25]:
df=pd.concat([df_train,df_test])

## Обучаем наш алгоритм


### Векторизуем сообщения тренировочного датасета

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
unrel_list=["алматы", "ао", "астана", "казахстан","казахстане", "казахстана","казахстанской", "енпф", "тенге", "2015", "2016", "2017", "kz"]
vectorizer = TfidfVectorizer(encoding='utf-8', max_features=100, stop_words=unrel_list)
vectors = vectorizer.fit_transform(df.text)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
words_df.head()

,10,12,20,банк,банка,более,будет,будут,был,были,было,во,время,все,всего,всех,год,года,году,государства,декабря,директор,для,до,его,ее,если,есть,еще,же,за,заместитель,из,или,им,их,как,когда,компании,компания,...,от,по,под,после,президент,президента,пресс,при,работы,развития,рамках,республики,рк,сегодня,сейчас,со,средств,страны,так,также,тем,то,того,только,том,тоо,тыс,тысяч,уже,управления,фонда,через,что,чтобы,экономики,это,этого,этом,является,января
0,0.000000,0.000000,0.000000,0.000000,0.568593,0.00000,0.000000,0.0,0.000000,0.000000,0.233381,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.163412,0.000000,0.000000,0.290049,0.157128,0.163649,0.000000,0.00000,0.146077,0.134139,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.094628,0.241989,0.000000,0.000000,0.0,0.0,0.123592,0.000000,0.000000,0.000000,0.000000,0.000000,0.109266,0.128506,0.000000,0.000000,0.000000,0.0,0.000000,0.091186,0.000000,0.117561,0.000000,0.0,0.000000,0.149177,0.000000,0.0,0.0,0.000000,0.0,0.0,0.438986,0.0,0.0,0.097940,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.000000,0.160155,0.616708,0.00000,0.000000,0.0,0.136193,0.000000,0.126565,0.00000,0.249701,0.122786,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101345,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0,0.184436,0.000000,0.000000,0.131278,0.098060,0.0,0.000000,0.000000,...,0.102635,0.262466,0.000000,0.000000,0.0,0.0,0.268100,0.000000,0.000000,0.000000,0.000000,0.000000,0.118512,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.171184,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.139491,0.000000,0.121442,0.000000,0.000000,0.22654,0.155265,0.0,0.000000,0.000000,0.033615,0.03775,0.033159,0.032611,0.080323,0.000000,0.036784,0.023537,0.090578,0.042431,0.000000,0.000000,0.047142,0.026916,0.03172,0.000000,0.077282,0.07578,0.03914,0.075765,0.261584,0.0,0.171446,0.073467,0.000000,0.034866,0.078132,0.0,0.076442,0.088537,...,0.081777,0.156845,0.040872,0.038666,0.0,0.0,0.000000,0.087474,0.000000,0.000000,0.000000,0.000000,0.000000,0.037018,0.122561,0.033374,0.000000,0.0,0.000000,0.026268,0.119045,0.067731,0.070434,0.0,0.034023,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.189685,0.0,0.0,0.141066,0.000000,0.117115,0.0,0.0
3,0.000000,0.462015,0.064687,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.116255,0.214862,0.00000,0.000000,0.000000,0.000000,0.062748,0.000000,0.376113,0.144743,0.000000,0.000000,0.000000,0.000000,0.086024,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.038001,0.0,0.039138,0.117398,0.000000,0.000000,0.041618,0.0,0.000000,0.000000,...,0.000000,0.222787,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.065971,0.056224,0.120084,0.125825,0.000000,0.059154,0.000000,0.000000,0.133967,0.0,0.054789,0.041975,0.063411,0.000000,0.000000,0.0,0.000000,0.274679,0.075484,0.0,0.0,0.000000,0.0,0.0,0.033679,0.0,0.0,0.000000,0.000000,0.046787,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.092870,0.272405,0.000000,0.000000,0.000000,0.000000,0.000000,0.104667,0.163516,0.062241,0.07335,0.000000,0.000000,0.00000,0.00000,0.000000,0.054990,0.0,0.283180,0.000000,0.105133,0.322499,0.120448,0.0,0.000000,0.000000,...,0.126068,0.362688,0.000000,0.000000,0.0,0.0,0.164655,0.404552,0.000000,0.000000,0.347541,0.000000,0.072785,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.060741,0.091760,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.146210,0.0,0.0,0.000000,0.087889,0.135408,0.0,0.0


### Инициализируем переменные

`X`: признаки.

`y`: метки.

In [28]:
X = words_df
y = df.polarity

### Выбираем алгоритм


In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

### Тренируем наш алгоритм


In [30]:
%%time
# Create and train a logistic regression
logreg = LogisticRegression(C=1e9, solver='lbfgs', max_iter=10000)
logreg.fit(X, y)

CPU times: user 1.13 s, sys: 497 ms, total: 1.63 s
Wall time: 897 ms


In [31]:
%%time
# Create and train a random forest classifier
forest = RandomForestClassifier(n_estimators=50)
forest.fit(X, y)

CPU times: user 1.88 s, sys: 85.4 ms, total: 1.97 s
Wall time: 1.88 s


In [32]:
%%time
# Create and train a linear support vector classifier (LinearSVC)
svc = LinearSVC()
svc.fit(X, y)

CPU times: user 252 ms, sys: 0 ns, total: 252 ms
Wall time: 254 ms


In [33]:
%%time
# Create and train a multinomial naive bayes classifier (MultinomialNB)
bayes = MultinomialNB()
bayes.fit(X, y)

CPU times: user 11 ms, sys: 3.32 ms, total: 14.3 ms
Wall time: 16.6 ms


In [34]:
pd.set_option("display.max_colwidth", 200)
unknown = df_test.drop(columns="polarity")

,text
0,Доброе утро! Смоленская АЭС. Матвеева
1,"Доброе утро, коллеги)"
2,"Всем привет. НВ АЭС, Чернышов Саша тут"
3,"Роман Топорков, Белоярская АЭС"
4,"Здравствуйте, Леонова Юлия, Смоленская АЭС"
...,...
113,"А почему власти из Москвы ничего не предпринимают. На федеральных каналах нет данной информации? По телевизору идет программа ""Давай поженимся"". Скрывают?!"
114,"Под красивыми словами о ""команде"" скрывается поощрение безответственности. В бою команды отдает один человек. ЧС - это бой. Должен быть четкий чек-лист четко расписано при каких показателях предпр..."
115,"Все понятно. Как ни пиши, что ни делай – вам все не так. Принял бы один человек решение – усомнились бы в его компетентности, правоте, дееспособности в конце концов. Команда – тоже все не то. Не..."
116,"Регламены (или, как сейчас модно говорить, чек-листы) имеются. Уровень для объявления эвакуции не достигнут. Поэтому её и не объявляют."


In [35]:
print(vectorizer.get_feature_names_out())

['10' '12' '20' 'банк' 'банка' 'более' 'будет' 'будут' 'был' 'были' 'было'
 'во' 'время' 'все' 'всего' 'всех' 'год' 'года' 'году' 'государства'
 'декабря' 'директор' 'для' 'до' 'его' 'ее' 'если' 'есть' 'еще' 'же' 'за'
 'заместитель' 'из' 'или' 'им' 'их' 'как' 'когда' 'компании' 'компания'
 'которые' 'который' 'лет' 'между' 'министр' 'млн' 'млрд' 'может' 'мы'
 'на' 'назад' 'нас' 'национальной' 'не' 'необходимо' 'но' 'об' 'области'
 'он' 'они' 'от' 'по' 'под' 'после' 'президент' 'президента' 'пресс' 'при'
 'работы' 'развития' 'рамках' 'республики' 'рк' 'сегодня' 'сейчас' 'со'
 'средств' 'страны' 'так' 'также' 'тем' 'то' 'того' 'только' 'том' 'тоо'
 'тыс' 'тысяч' 'уже' 'управления' 'фонда' 'через' 'что' 'чтобы'
 'экономики' 'это' 'этого' 'этом' 'является' 'января']


## Векторизуем сообщения тестировочного датасета


In [36]:
unknown_vectors = vectorizer.transform(unknown.text)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,12,20,банк,банка,более,будет,будут,был,были,было,во,время,все,всего,всех,год,года,году,государства,декабря,директор,для,до,его,ее,если,есть,еще,же,за,заместитель,из,или,им,их,как,когда,компании,компания,...,от,по,под,после,президент,президента,пресс,при,работы,развития,рамках,республики,рк,сегодня,сейчас,со,средств,страны,так,также,тем,то,того,только,том,тоо,тыс,тысяч,уже,управления,фонда,через,что,чтобы,экономики,это,этого,этом,является,января
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Predict using all our models. 

# Logistic Regression predictions + probabilities
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
unknown['pred_logreg_proba'] = logreg.predict_proba(unknown_words_df)[:,1]

# Random forest predictions + probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['pred_forest_proba'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC predictions
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes predictions + probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['pred_bayes_proba'] = bayes.predict_proba(unknown_words_df)[:,1]

In [38]:
unknown

,text,pred_logreg,pred_logreg_proba,pred_forest,pred_forest_proba,pred_svc,pred_bayes,pred_bayes_proba
0,Доброе утро! Смоленская АЭС. Матвеева,0,0.517758,0,0.576167,0,0,0.491111
1,"Доброе утро, коллеги)",0,0.517758,0,0.576167,0,0,0.491111
2,"Всем привет. НВ АЭС, Чернышов Саша тут",0,0.517758,0,0.576167,0,0,0.491111
3,"Роман Топорков, Белоярская АЭС",0,0.517758,0,0.576167,0,0,0.491111
4,"Здравствуйте, Леонова Юлия, Смоленская АЭС",0,0.517758,0,0.576167,0,0,0.491111
...,...,...,...,...,...,...,...,...
113,"А почему власти из Москвы ничего не предпринимают. На федеральных каналах нет данной информации? По телевизору идет программа ""Давай поженимся"". Скрывают?!",-1,0.361037,-1,0.140000,-1,0,0.495223
114,"Под красивыми словами о ""команде"" скрывается поощрение безответственности. В бою команды отдает один человек. ЧС - это бой. Должен быть четкий чек-лист четко расписано при каких показателях предпр...",0,0.451036,0,0.737829,0,0,0.484294
115,"Все понятно. Как ни пиши, что ни делай – вам все не так. Принял бы один человек решение – усомнились бы в его компетентности, правоте, дееспособности в конце концов. Команда – тоже все не то. Не...",-1,0.413412,0,0.880000,0,0,0.495561
116,"Регламены (или, как сейчас модно говорить, чек-листы) имеются. Уровень для объявления эвакуции не достигнут. Поэтому её и не объявляют.",0,0.768736,1,0.320000,0,0,0.547907


## Тесты обученной модели


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
%%time

print("Training logistic regression")
logreg.fit(X_train, y_train)

print("Training random forest")
forest.fit(X_train, y_train)

print("Training SVC")
svc.fit(X_train, y_train)

print("Training Naive Bayes")
bayes.fit(X_train, y_train)

Training logistic regression
Training random forest
Training SVC
Training Naive Bayes
CPU times: user 2.44 s, sys: 498 ms, total: 2.94 s
Wall time: 2.28 s


### Матрицы ошибок

In [44]:
from sklearn.metrics import confusion_matrix

#### Логистическая регрессия

In [45]:
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'neutral', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted neutral,Predicted positive
Is negative,126,240,38
Is neutral,82,707,238
Is positive,20,300,345


#### Случайный лес

In [46]:
y_true = y_test
y_pred = forest.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted neutral,Predicted positive
Is negative,117,254,33
Is neutral,57,747,223
Is positive,9,301,355


#### SVC

In [47]:
y_true = y_test
y_pred = svc.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted neutral,Predicted positive
Is negative,87,275,42
Is neutral,52,722,253
Is positive,11,294,360


#### Наивный Байес

In [48]:
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted neutral,Predicted positive
Is negative,0,394,10
Is neutral,0,940,87
Is positive,0,512,153
